# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [24]:
from azureml.core import Workspace, Dataset, Experiment
from azureml.core.webservice import Webservice
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import logging
import joblib


## Dataset

### Overview
The dataset is a compilation of social and economic information from all neighborhoods in the city of Madrid. We will try to create a model that predicts the average income of a neighborhood based on the rest of indicators.

The dataset was gathered from Madrid's [open data website][https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=71359583a773a510VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default] and was transformed to a tabular dataset.

In [5]:
# ws = Workspace.from_config()

subscription_id = '9a8ef160-b36c-4d1c-95d2-b381d53baaa3'
resource_group = 'rg-bigdatanetworks-uad-pro'
workspace_name = 'aml-BigDataNetworksuad-pro'

ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name='panel_indicadores_distritos_barrios_2022')
df = dataset.to_pandas_dataframe()
df.head(5)

,Barrio,Año medio de contrucción de inmuebles de uso residencial,Apartamentos Municipales para Mayores,Asociaciones (Sección 1ª),Asociaciones culturales y casas regionales,Asociaciones de mujeres,Asociaciones vecinales,Bibliotecas Comunidad Madrid,Bibliotecas Municipales,Campos de fútbol 11,...,Tasa absoluta de paro registrado Mujeres,Tasa bruta de natalidad (‰),Tasa de crecimiento demográfico (porcentaje),Tasa de desempleo en hombres de 16 a 24 años,Tasa de desempleo en hombres de 25 a 44 años,Tasa de desempleo en hombres de 45 a 64 años,Tasa de desempleo en mujeres de 16 a 24 años,Tasa de desempleo en mujeres de 25 a 44 años,Tasa de desempleo en mujeres de 45 a 64 años,Total hogares
0,11,1928,1,49,22,2,1,0,1,0,...,7.24,6.4,0.0,3.00,5.01,11.38,1.48,6.28,9.96,11613
1,12,1927,0,98,28,4,9,1,0,0,...,8.29,6.3,0.4,3.71,5.31,10.54,4.23,6.96,11.46,22614
2,13,1923,0,31,10,1,1,0,0,0,...,5.93,5.9,0.2,0.90,3.45,8.73,0.79,4.91,8.78,5488
3,14,1929,0,36,10,4,1,0,1,0,...,5.26,6.8,1.0,1.28,3.76,9.13,0.64,3.56,8.85,8927
4,15,1930,0,46,16,1,2,0,1,0,...,6.32,5.6,0.0,2.63,4.57,9.34,1.20,4.59,10.36,16967


In [6]:
# choose a name for experiment
experiment_name = 'aml-barrios-madrid-2022'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

The configuration is pretty standard. I chose low numbers for the timeout as the performance of the model is not that relevant and we can skip models that take too long to train. The `max_concurrent_iterations` was chosen to max out the nodes available in the compute cluster, and the `max_cores_per_iteration` parameter also reflects that the maximum number of cores on each node should be used.

I chose 3 cross-validations as I always like to use CV when training models. Ideally I would choose more (like 5) but given the sample size we have isn't too high (after all there's a limited number of neighborhoods in Madrid), I chose 3.
 
The optimization metric is NRMSE as I want to penalize models whose predictions have extreme errors (even if the average deviations aren't that bad).




In [7]:
from azureml.train.automl.utilities import get_primary_metrics
get_primary_metrics('regression')

['normalized_mean_absolute_error',
 'spearman_correlation',
 'r2_score',
 'normalized_root_mean_squared_error']

In [8]:
automl_settings = {
    'experiment_timeout_hours': 0.25,
    'enable_early_stopping':True,
    'max_concurrent_iterations': 8,
    'max_cores_per_iteration': -1,
    'featurization': 'auto',
    'verbosity': logging.INFO
}

automl_config = AutoMLConfig(task='regression',
                             training_data=dataset,
                             label_column_name='Renta neta media anual de los hogares (Urban Audit)',
                             iterations=30,
                             iteration_timeout_minutes=5,
                             primary_metric='normalized_root_mean_squared_error',
                             n_cross_validations=3,
                             compute_target=ComputeTarget(workspace=ws, name='CLBigDataNetworksuadDS5V2pro'),
                             debug_log='automl_errors.log',
                             **automl_settings
                            )

In [9]:
# TODO: Submit your experiment
# runs = list(experiment.get_runs())

# if len(runs) > 0: # We had previously launched this experiment
#     remote_run=runs[0]
# else:
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on CLBigDataNetworksuadDS5V2pro with default configuration
Running on remote compute: CLBigDataNetworksuadDS5V2pro


Experiment,Id,Type,Status,Details Page,Docs Page
aml-barrios-madrid-2022,AutoML_a8b0dd36-7a84-439c-8beb-4fe5cff347b7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

************************************************************

In [12]:
type(remote_run)

azureml.train.automl.run.AutoMLRun

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [33]:
# Retrieve and save your best automl model.
best_run, fitted_model = remote_run.get_output()
print(f'Best run: {best_run}')
print(f'Model: {fitted_model}')

Best run: Run(Experiment: aml-barrios-madrid-2022,
Id: AutoML_a8b0dd36-7a84-439c-8beb-4fe5cff347b7_29,
Type: azureml.scriptrun,
Status: Completed)
Model: RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='regression', working_dir='/mnt/batch/ta...
                                             StackEnsembleRegressor(base_learners=[('26', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('gradientboostingregressor', GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None, learning_rate=0.01, loss='huber', max_depth=2, max_features=0.2, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_

In [18]:
joblib.dump(fitted_model, 'automl_model.pkl')

['automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
model_name = best_run.properties['model_name']
description = 'Best model trained from AutoML'
tags = None
model = remote_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

# The deployment was performed using the GUI as we were taught in the previous lesson.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data =  {
  "Inputs": {
    "data": [
      {
        "Barrio": 5,
        "Año medio de contrucción de inmuebles de uso residencial": 1999,
        "Apartamentos Municipales para Mayores": 1,
        "Asociaciones (Sección 1ª)": 2,
        "Asociaciones culturales y casas regionales": 0,
        "Asociaciones de mujeres": 0,
        "Asociaciones vecinales": 1,
        "Bibliotecas Comunidad Madrid": 1,
        "Bibliotecas Municipales": 1,
        "Campos de fútbol 11": 1,
        "Centro de Día de Atención a Niños y Niñas (de 3 a 12 años)": 0,
        "Centros de Adolescentes y Jóvenes (ASPA)": 0,
        "Centros de Apoyo a las Familias (CAF)": 0,
        "Centros de Atención a la Infancia (CAI)": 0,
        "Centros de Atención a las Adicciones (CAD y CCAD)": 0,
        "Centros de Día de Alzheimer y Físicos": 0,
        "Centros de Servicios Sociales": 0,
        "Centros deportivos Municipales": 2,
        "Centros Municipales de Mayores": 0,
        "Centros Municipales de Salud Comunitaria (CMSC)": 0,
        "Centros para personas sin hogar": 0,
        "Centros y Espacios Culturales": 0,
        "Colegios Públicos Infantil y Primaria": 5,
        "Duración media del crédito (meses) en transacción de vivienda": 200,
        "Edad media de la población": 48,
        "Escuelas Infantiles Municipales": 5,
        "Espacios de Igualdad": 0,
        "Espacios de Ocio para Adolescentes (El Enredadero)": 0,
        "Etapas educativas. Total niñas": 400,
        "Etapas educativas. Total niños": 400,
        "Fundaciones (Sección 2ª)": 2,
        "Hogares con un hombre solo mayor de 65 años": 200,
        "Hogares con una mujer sola mayor de 65 años": 200,
        "Hogares monoparentales: un hombre adulto con uno o más menores": 0,
        "Hogares monoparentales: una mujer adulta con uno o más menores": 0,
        "Índice de dependencia (Población de 0-15 + población 65 años y más / Pob. 16-64)": 40,
        "Instalaciones deportivas básicas": 1,
        "Mercados Municipales": 1,
        "Número de inmuebles de uso residencial": 1500,
        "Número Habitantes": 3000,
        "Paro registrado (número de personas registradas en SEPE en Febrero 2022)": 5,
        "Paro registrado (número de personas registradas en SEPE en Febrero 2022) Hombres": 5,
        "Paro registrado (número de personas registradas en SEPE en Febrero 2022) Mujeres": 5,
        "Pensión media mensual  Mujeres": 1800,
        "Pensión media mensual Hombres": 2100,
        "Personas con nacionalidad española": 2800,
        "Personas con nacionalidad española Hombres": 1400,
        "Personas con nacionalidad española Mujeres": 1400,
        "Personas con nacionalidad extranjera": 200,
        "Personas con nacionalidad extranjera Hombres": 100,
        "Personas con nacionalidad extranjera Mujeres": 100,
        "Piscinas cubiertas": 1,
        "Piscinas de verano": 1,
        "Pista de atletismo": 0,
        "Población de 0 a 14 años": 800,
        "Población de 15 a 29 años": 700,
        "Población de 30 a 44  años": 500,
        "Población de 45 a 64 años": 200,
        "Población de 65 a 79 años":200,
        "Población de 65 años y más": 200,
        "Población de 80 años y más": 400,
        "Población densidad (hab./Ha.)": 300,
        "Población en etapa educativa (Población de 3 a 16 años -16 no incluidos)": 0,
        "Población en etapa educativa de 0 a 2 años": 200,
        "Población en etapa educativa de 12 a 15 años": 200,
        "Población en etapa educativa de 3 a 5 años": 200,
        "Población en etapa educativa de 6 a 11 años": 200,
        "Población en etapas educativas": 800,
        "Población Hombres": 1500,
        "Población infantil femenina en etapa educativa de 0 a 2 años": 100,
        "Población infantil femenina en etapa educativa de 12 a 15 años": 100,
        "Población infantil femenina en etapa educativa de 3 a 5 años": 100,
        "Población infantil femenina en etapa educativa de 6 a 11 años": 100,
        "Población infantil masculina en etapa educativa de 0 a 2 años": 100,
        "Población infantil masculina en etapa educativa de 12 a 15 años": 100,
        "Población infantil masculina en etapa educativa de 3 a 5 años": 100,
        "Población infantil masculina en etapa educativa de 6 a 11 años": 100,
        "Población mayor/igual  de 25 años  con estudios superiores, licenciatura, arquitectura, ingeniería sup., estudios sup. no universitarios, doctorado,  postgraduado": 80,
        "Población mayor/igual  de 25 años  que no sabe leer ni escribir o sin estudios": 0,
        "Población mayor/igual  de 25 años con Bachiller Elemental, Graduado Escolar, ESO, Formación profesional 1º grado": 5,
        "Población mayor/igual  de 25 años con enseñanza primaria incompleta": 0,
        "Población mayor/igual  de 25 años con Formación profesional 2º grado, Bachiller Superior o BUP": 5,
        "Población mayor/igual  de 25 años con Nivel de estudios desconocido y/o no consta": 0,
        "Población mayor/igual  de 25 años con titulación media, diplomatura, arquitectura o ingeniería técnica": 10,
        "Población Mujeres": 1500,
        "Proporción de envejecimiento (Población mayor de 65 años/Población total)": 20,
        "Proporción de juventud (Población de 0-15 años/Población total) porcentaje": 20,
        "Proporción de personas migrantes (Población extranjera menos UE y resto países de OCDE / Población total)": 5,
        "Proporción de sobre-envejecimiento (Población mayor de 80 años/ Población mayor de 65 años)": 15,
        "Residencias para personas Mayores": 2,
        "Superficie (Ha.)": 150,
        "Superficie media de la vivienda (m2) en transacción": 120,
        "Tamaño medio del hogar": 3.22,
        "Tasa absoluta de paro registrado (Febrero  2022)": 3,
        "Tasa absoluta de paro registrado Hombres": 3,
        "Tasa absoluta de paro registrado Mujeres": 4,
        "Tasa bruta de natalidad (‰)": 12.0,
        "Tasa de crecimiento demográfico (porcentaje)": 10,
        "Tasa de desempleo en hombres de 16 a 24 años": 9.4,
        "Tasa de desempleo en hombres de 25 a 44 años": 4.4,
        "Tasa de desempleo en hombres de 45 a 64 años": 7.1,
        "Tasa de desempleo en mujeres de 16 a 24 años": 9.5,
        "Tasa de desempleo en mujeres de 25 a 44 años": 5.2,
        "Tasa de desempleo en mujeres de 45 a 64 años": 7.3,
        "Total hogares": 3500
      }
    ]
  },
  "GlobalParameters": 0.0
}

body = str.encode(json.dumps(data))

url = 'http://77c2d7fe-72d3-44d7-8be8-2b9ad0486b75.westeurope.azurecontainer.io/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'uEG6CQYUeQic60N2XMRCXQ4jgcSxLNpV'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

URLError: <urlopen error [Errno 110] Connection timed out>

Unfortunately the AzureML subscription I use does not have public internet access, so I cannot reach the endpoint within this notebook. I have tried in a local python install to launch an inference request to this endpoint, and a result was received successfully.

`b'{"Results": [42890.21257466434]}'`

TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
name = "barrios-madrid-endpoint"

# load existing web service
service = Webservice(name=name, workspace=ws)

# enable application insight
# service.update(enable_app_insights=True) # already enabled with the GUI

logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2023-10-05T12:15:57,860027300+00:00 - rsyslog/run 
2023-10-05T12:15:57,860027400+00:00 - iot-server/run 
2023-10-05T12:15:57,883200700+00:00 - gunicorn/run 
2023-10-05T12:15:57,886615300+00:00 | gunicorn/run | 
2023-10-05T12:15:57,891585300+00:00 | gunicorn/run | ###############################################
2023-10-05T12:15:57,897506400+00:00 - nginx/run 
2023-10-05T12:15:57,902610100+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-10-05T12:15:57,911890100+00:00 | gunicorn/run | ###############################################
2023-10-05T12:15:57,922867500+00:00 | gunicorn/run | 
2023-10-05T12:15:57,940598500+00:00 | gunicorn/run | 
2023-10-05T12:15:57,966285600+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230120.v2
2023-10-05T12:15:57,974097800+00:00 | gunicorn/run | 
2023-10-05T12:15:57,977480200+00:00 | gunicorn/run | 
2023-10-05T12:15:57,983705800+00:00 | gunicorn/run | PATH environment variable: /azureml-env

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
